In [3]:
!pip install strands-agents[mistral] python-dotenv dictdiffer

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [strands-agents]m [strands-agents]gs]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.


In [1]:
import json
import os
import sys
import dotenv

import pprint
import copy

import yaml
from pathlib import Path, PosixPath
from typing import Dict, List, Tuple, TypeVar
from tqdm import tqdm

# Pydantic for structured data
from pydantic import BaseModel, Field

# Strands for AI agents
from strands.agent import Agent
from strands.models.mistral import MistralModel

from collections import Counter

sys.path.append('..')

from src.utils import (
    save_json,
    read_json,
    load_file_content,
    get_job_paths,
    get_training_paths,
    sanity_check,
	chat_with_persona,
    track_api_call,  # Cost tracking from utils
    print_cost_summary,  # Cost summary from utils
    reset_cost_tracker  # Reset cost tracker from utils
)

from src.my_utils import (
    display_markdown_file,
    call_mistral,
    get_agent,
    batch_extract,
    compute_stat_for_multi_items
)

from src.models.activity_domain_info import ActivityDomainLabelingReply
from src.models.job_info import (
    JobInfo
)

from src.models.skill_info import(
    SkillLabelingReply,
    SkillDomainLabelingReply
)

from src.prompts.job_extraction_prompt import(
    DOMAIN_LABELING_PROMPT
)

# Load API key from .env file
dotenv.load_dotenv("../env")

# Check if we're good to go
if not os.getenv("MISTRAL_API_KEY"):
    print("❌ No MISTRAL_API_KEY found!")
    print("Create an env file with your API key")
else:
    print("✅ API key found, we're ready to roll")

✅ API key found, we're ready to roll


In [2]:
DATA_JOBS_DIR = Path('../data_jobs')
DATA_ACTIVITIES_DOMAINS_DIR = Path('../data_activities_domains')

In [3]:
with open("../src/config.yaml", "r") as f:
    config = yaml.safe_load(f)

job_data_version = config["job_data_version"]
print(f"job_data_version version: {job_data_version}")

activity_domains_version = config["activity_domains_version"]
print(f"activity_domains_version version: {activity_domains_version}")

job_data_version version: v4
activity_domains_version version: v4


In [4]:
# Load jobs data
filename = f"clusterized_jobs_{job_data_version}.json"
jobs_save_path = DATA_JOBS_DIR / filename
jobs_data = read_json(jobs_save_path)

# Convert to JobInfo objects
jobs_info = {
    job_id: JobInfo.model_validate_json(data)
    for job_id, data in jobs_data.items()
}

print(f"✅ Loaded {len(jobs_info)} jobs")
print("\n" + "="*50)

✅ Loaded 200 jobs



# Functions definition

In [5]:
def compute_activity_domain_label(
    jobs_list,
    model: str = "mistral-medium-latest",
    print_prompt: bool = False
) -> ActivityDomainLabelingReply:

    jobs_str = ""
    for id in jobs_list:
        job_info = jobs_info[id]
        jobs_str += "- " + job_info.title + ': ' + job_info.job_description + "\n"

    prompt = DOMAIN_LABELING_PROMPT.format(
        jobs_description=jobs_str
    )

    if print_prompt is True:
        print(prompt)

    extraction_agent = get_agent(model_id=model)
    result = extraction_agent.structured_output(output_model=ActivityDomainLabelingReply, prompt=prompt)

    return result

# Build structured hierarchy

In [6]:
# print(jobs_data)

jobs_map = {}
for id in jobs_data:
    job_info = jobs_info[id]

    activity_domains = job_info.domains
    for domain in activity_domains:
        if domain not in jobs_map:
            jobs_map[domain] = []

        jobs_map[domain].append(id)

print(jobs_map)

{'CLUSTER_7': ['j0', 'j1', 'j10', 'j12', 'j13', 'j14', 'j15', 'j16', 'j19', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7', 'j8', 'j9'], 'CLUSTER_9': ['j100', 'j101', 'j102', 'j103', 'j104', 'j105', 'j106', 'j107', 'j108', 'j109'], 'CLUSTER_13': ['j11', 'j17', 'j18', 'j90', 'j91', 'j92', 'j93', 'j94', 'j95', 'j96', 'j97', 'j98', 'j99'], 'CLUSTER_4': ['j110', 'j111', 'j112', 'j113', 'j114', 'j115', 'j116', 'j117', 'j118', 'j119', 'j120', 'j121', 'j122', 'j123', 'j124', 'j125', 'j126', 'j127', 'j128', 'j129', 'j180', 'j184', 'j187'], 'CLUSTER_12': ['j130', 'j131', 'j132', 'j133', 'j134', 'j135', 'j136', 'j137', 'j138', 'j150', 'j151', 'j153', 'j154', 'j155', 'j156', 'j157', 'j158', 'j159', 'j182', 'j183', 'j185', 'j186'], 'CLUSTER_5': ['j139', 'j80', 'j81', 'j82', 'j83', 'j84', 'j85', 'j86', 'j87', 'j88', 'j89'], 'CLUSTER_8': ['j140', 'j141', 'j142', 'j143', 'j144', 'j145', 'j146', 'j147', 'j148', 'j149'], 'CLUSTER_11': ['j152', 'j181', 'j188', 'j189', 'j40', 'j41', 'j42', 'j43', 'j44', 'j45', 'j46'

# Label Activity Domain fields

In [10]:
extended_jobs_map = {}

for domain in tqdm(jobs_map):        
    result = compute_activity_domain_label(
        jobs_map[domain],
        model="mistral-medium-latest",
        print_prompt=False)

    print(f"LABEL : {result.activity_domain_label}")
    print(f"DESCRIPTION : {result.description}")
    print(f"RATIONALE : {result.rationale}")
    print("")

    job_map = {}
    job_map['description'] = result.description
    job_map['job_ids'] = jobs_map[domain]
    
    label = result.activity_domain_label

    # extended_jobs_map[label] = jobs_map[domain]
    extended_jobs_map[label] = job_map

print(extended_jobs_map)

job_data_version = 'v5'

filename = f"map_clusters_jobs_{job_data_version}.json"
save_path = DATA_JOBS_DIR / filename
save_json(save_path, extended_jobs_map)

  7%|▋         | 1/15 [00:03<00:52,  3.74s/it]

LABEL : Financial Operations And Regulatory Compliance
DESCRIPTION : Covers accounting, banking, and insurance sectors focusing on compliance, risk management, and financial operations.
RATIONALE : The proposed domain name captures the core activities present across the listed jobs, which primarily revolve around financial operations, regulatory compliance, and risk management. The primary industry sectors identified in the job data are banking, accounting, and insurance.  

Outliers: None of the jobs appear to be significant outliers in function, as they all relate to financial operations, compliance, or risk management within the specified sectors.



 13%|█▎        | 2/15 [00:06<00:43,  3.32s/it]

LABEL : Legal Support And Compliance Analysis
DESCRIPTION : Focuses on legal research, compliance, and document drafting in sectors like law and corporate governance.
RATIONALE : The proposed domain name captures the core activities of legal research, compliance, and support present across all listed jobs. The description highlights the primary sectors identified, such as law and corporate governance, while also mentioning the focus areas of legal research and document drafting. No significant outliers were detected in the job functions, and the description remains concise and informative.



 20%|██        | 3/15 [00:12<00:51,  4.25s/it]

...

LABEL : Design Research And Innovation Management
DESCRIPTION : Focuses on user research and innovation management primarily in product development and design sectors.
RATIONALE : The proposed domain name captures the core activities of design research and innovation management evident in the job listings. The description highlights the primary sectors of product development and design. No significant outliers were detected as all jobs revolve around design research, user experience studies, and innovation management.



 93%|█████████▎| 14/15 [00:59<00:04,  4.21s/it]

LABEL : Fiber And Paper Manufacturing Operations
DESCRIPTION : This domain focuses on roles within the fiber and paper manufacturing and processing sectors.
RATIONALE : The proposed domain name captures the core activities and industry sectors identified in the job data, which primarily revolve around fiber and paper manufacturing and processing. The jobs listed are mainly centered on production optimization, quality control, research and development, and sustainable practices within these sectors. No significant outliers were detected in the job functions provided.



100%|██████████| 15/15 [01:05<00:00,  4.35s/it]

LABEL : Food Production And Quality Management
DESCRIPTION : This domain covers roles in the Food Industry focusing on production, quality, safety, and innovation.
RATIONALE : The proposed domain name captures the core activities present in the job listings, which primarily revolve around food production, quality assurance, and safety management. The description explicitly mentions the primary industry sector identified, which is the Food Industry. The jobs listed are mostly aligned with these activities, ensuring the domain name is both generic enough to fit all jobs and specific enough to capture their unique character. No significant outliers were detected in terms of function, as all roles are closely related to food production and quality management.

{'Financial Operations And Regulatory Compliance': {'description': 'Covers accounting, banking, and insurance sectors focusing on compliance, risk management, and financial operations.', 'job_ids': ['j0', 'j1', 'j10', 'j12', 'j13', '

# Add job title to map for better analysis

In [34]:
filename = f"map_clusters_jobs_{job_data_version}.json"
save_path = DATA_JOBS_DIR / filename
jobs_map = read_json(save_path)

# print(jobs_map)

In [38]:
extended_jobs_map = {}
for domain in jobs_map:
    extended_jobs_map[domain] = copy.deepcopy(jobs_map[domain])
    # print(extended_jobs_map)
    # print(type(jobs_map[domain]['job_ids']))
    # print(jobs_map[domain]['job_ids'])
    extended_jobs_map[domain]['job_ids'] = {}
    for id in jobs_map[domain]['job_ids']:
        # print(jobs_map[domain])
        job_info = jobs_info[id]
        extended_jobs_map[domain]['job_ids'][id] = job_info.title

# print(extended_jobs_map)
filename = f"map_clusters_jobs_{job_data_version}.json"
save_path = DATA_JOBS_DIR / filename
save_json(save_path, extended_jobs_map)

# Apply clusters labels to job list

In [42]:
filename = f"map_clusters_jobs_{job_data_version}.json"
save_path = DATA_JOBS_DIR / filename
jobs_map = read_json(save_path)

for domain in jobs_map:
    for id in jobs_map[domain]['job_ids']:
        job_data_dict = json.loads(jobs_data[id])
        job_data_dict['domain'] = domain
        job_data_dict['domains'] = [domain]
        job_data = json.dumps(job_data_dict, ensure_ascii=False)
        jobs_data[id] = job_data

filename = f"extended_jobs_{job_data_version}.json"
jobs_save_path = DATA_JOBS_DIR / filename
save_json(jobs_save_path, jobs_data)